In [ ]:
import math
import copy
import random


In [ ]:
def misplaced_tiles(puzzle):
  misplaced = 0;
  for i in range(len(puzzle)):
    for j in range(len(puzzle[i])):
      if (puzzle[i][j] != (i*len(puzzle) + j)):
        misplaced +=1
  return misplaced

def manhatten_distance(puzzle):
  manhatten = 0
  vertical,horizontal = 0,0

  for i in range(len(puzzle)):
    for j in range(len(puzzle[i])):
      vertical    = abs(puzzle[i][j] // len(puzzle)-i)
      horizontal  = abs(puzzle[i][j] % len(puzzle[0]) -j)
      manhatten += vertical + horizontal
  return manhatten



In [ ]:
def moves(puzzle):
  move = []
  zero_position = ()
  for i in range(len(puzzle)):
    for j in range(len(puzzle[i])):
      if(puzzle[i][j] == 0):
        zero_position = (i,j)
        if(i < len(puzzle)-1):
          move.append((1, 0))  #checking down

        if(i > 0):
          move.append((-1, 0))  #checking up

        if(j < len(puzzle[i])-1):
          move.append((0, 1))  #checking right

        if(j > 0):
          move.append((0, -1))  #checking left
  return move,zero_position

In [ ]:
def read_matrix_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Assuming the matrix is represented line by line in the file
        matrix = [list(map(int, line.strip().split())) for line in lines]
        return matrix

    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

file_path = 'input-2.txt'
start_state = read_matrix_from_file(file_path)

if start_state:
    print(f"Start State: {start_state}")

goal_state  = [[0,1,2],
               [3,4,5],
               [6,7,8]]

Start State: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]


In [ ]:
def get_lowest_cost_successor(curr, curr_h, heuristic):
  neighbour = copy.deepcopy(curr)
  move, zero = moves(curr)
  min_h = curr_h
  for m in move:
    n = copy.deepcopy(neighbour)
    n[zero[0]][zero[1]] = neighbour[m[0]+zero[0]][m[1]+zero[1]]
    n[m[0]+zero[0]][m[1]+zero[1]] = 0
    h = heuristic(n)
    if(h < min_h):
      curr = copy.deepcopy(n)
      min_h = h
  return curr,min_h

def steepest_hill_climbing(state, goal, heuristic):
  currentState = state
  currentMinHeuristic = heuristic(currentState)
  itr = 0
  while(True):

    if(currentState == goal):
      print("Solution:", neighbour, " heuristic = ",currentMinHeuristic, " iteration = ",itr, "\n")
      return currentState,currentMinHeuristic,itr

    if(itr > 300):
      return currentState,currentMinHeuristic,itr

    neighbour,currentMinHeuristic = get_lowest_cost_successor(currentState,currentMinHeuristic,heuristic)
    print("Neighbour:", neighbour, " heuristic = ",currentMinHeuristic, " iteration = ",itr)

    if(neighbour == currentState):
      return currentState,currentMinHeuristic,itr
    currentState = copy.deepcopy(neighbour)

    itr += 1


In [ ]:
steepest_hill_climbing(start_state, goal_state, misplaced_tiles)

Neighbour: [[3, 1, 2], [6, 4, 5], [7, 8, 0]]  heuristic =  5  iteration =  0
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 0, 8]]  heuristic =  4  iteration =  1
Neighbour: [[3, 1, 2], [6, 4, 5], [0, 7, 8]]  heuristic =  3  iteration =  2
Neighbour: [[3, 1, 2], [0, 4, 5], [6, 7, 8]]  heuristic =  2  iteration =  3
Neighbour: [[0, 1, 2], [3, 4, 5], [6, 7, 8]]  heuristic =  0  iteration =  4
Solution: [[0, 1, 2], [3, 4, 5], [6, 7, 8]]  heuristic =  0  iteration =  5 



([[0, 1, 2], [3, 4, 5], [6, 7, 8]], 0, 5)

In [ ]:
steepest_hill_climbing(start_state, goal_state, manhatten_distance)

Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  heuristic =  8  iteration =  0


([[3, 1, 2], [6, 4, 0], [7, 8, 5]], 8, 0)

In [ ]:
def schedule(t):
    if t >= 500:
        return 0
    else:
        return 500 - t

def simulated_annealing(state, goal, schedule, heuristic):
  random.seed(100)
  curr = state

  for t in range(1, 301):
    h = heuristic(curr)
    T = schedule(t)

    if(T == 0):
      print("Solution:", curr, " h = ",h, " t= ",t)
      return curr, h, t

    if(curr == goal):
      print("Solution:", curr, " h = ",h, " t= ",t)
      return curr, h, t

    move,zero = moves(curr)
    r = random.randint(0,len(move)-1)

    next = copy.deepcopy(curr)
    next[zero[0]][zero[1]] = curr[move[r][0]+zero[0]][move[r][1]+zero[1]]
    next[move[r][0]+zero[0]][move[r][1]+zero[1]] = 0


    E = heuristic(curr) - heuristic(next)
    if(E > 0):
      print("Neighbour:", curr, " h= ",h, " t= ",t)
      curr = copy.deepcopy(next)
    elif(random.random() <= math.exp(E/T)):
      print("Neighbour:", curr, " h= ",h, " t= ",t)
      curr = copy.deepcopy(next)
    else:
      print("Neighbour:", curr, " h= ",h, " t= ",t)
      return curr, h, t
  return curr, heuristic(curr), 300

In [ ]:
print(simulated_annealing(start_state, goal_state, schedule, misplaced_tiles))

Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  6  t=  1
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 8, 0]]  h=  5  t=  2
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 0, 8]]  h=  4  t=  3
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 8, 0]]  h=  5  t=  4
Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  6  t=  5
Neighbour: [[3, 1, 2], [6, 0, 4], [7, 8, 5]]  h=  7  t=  6
Neighbour: [[3, 1, 2], [6, 8, 4], [7, 0, 5]]  h=  7  t=  7
Neighbour: [[3, 1, 2], [6, 0, 4], [7, 8, 5]]  h=  7  t=  8
Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  6  t=  9
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 8, 0]]  h=  5  t=  10
Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  6  t=  11
Neighbour: [[3, 1, 0], [6, 4, 2], [7, 8, 5]]  h=  7  t=  12
Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  6  t=  13
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 8, 0]]  h=  5  t=  14
Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  6  t=  15
Neighbour: [[3, 1, 0], [6, 4, 2], [7, 8, 5]]  h=  7  t=  16
Neighbour: [[3, 0, 1], [6, 4, 2], [7, 8, 5]]  h= 

In [ ]:
print(simulated_annealing(start_state, goal_state, schedule, manhatten_distance))

Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  8  t=  1
Neighbour: [[3, 1, 2], [6, 4, 5], [7, 8, 0]]  h=  8  t=  2
Neighbour: [[3, 1, 2], [6, 4, 0], [7, 8, 5]]  h=  8  t=  3
Neighbour: [[3, 1, 2], [6, 0, 4], [7, 8, 5]]  h=  8  t=  4
Neighbour: [[3, 1, 2], [6, 8, 4], [7, 0, 5]]  h=  10  t=  5
Neighbour: [[3, 1, 2], [6, 0, 4], [7, 8, 5]]  h=  8  t=  6
Neighbour: [[3, 1, 2], [0, 6, 4], [7, 8, 5]]  h=  8  t=  7
Neighbour: [[3, 1, 2], [6, 0, 4], [7, 8, 5]]  h=  8  t=  8
Neighbour: [[3, 0, 2], [6, 1, 4], [7, 8, 5]]  h=  8  t=  9
Neighbour: [[3, 2, 0], [6, 1, 4], [7, 8, 5]]  h=  10  t=  10
Neighbour: [[3, 2, 4], [6, 1, 0], [7, 8, 5]]  h=  12  t=  11
Neighbour: [[3, 2, 4], [6, 1, 5], [7, 8, 0]]  h=  12  t=  12
Neighbour: [[3, 2, 4], [6, 1, 5], [7, 0, 8]]  h=  10  t=  13
Neighbour: [[3, 2, 4], [6, 1, 5], [7, 8, 0]]  h=  12  t=  14
Neighbour: [[3, 2, 4], [6, 1, 0], [7, 8, 5]]  h=  12  t=  15
Neighbour: [[3, 2, 0], [6, 1, 4], [7, 8, 5]]  h=  10  t=  16
Neighbour: [[3, 0, 2], [6, 1, 4], [7, 8, 